In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob

from sklearn.utils.class_weight import compute_class_weight

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, VGG16, VGG19, InceptionV3, ResNet50, Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
plt.rcParams['figure.dpi'] = 300

# Prepare data for training and tuning set

## Initialize Parameters 

In [ ]:
input_shape = [299, 299]
batch_size = 32

train_dir = '/media/wf/Data/Diagnosis/train/'
tune_dir = '/media/wf/Data/Diagnosis/tune/'

## Data generator and data augment

In [ ]:
datagen_train = ImageDataGenerator(
      rescale=1./255,
      horizontal_flip=True,
      vertical_flip=True,
)

In [ ]:
datagen_tuning = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode='categorical')

In [ ]:
generator_tuning = datagen_tuning.flow_from_directory(directory=tune_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
steps_per_epoch = generator_train.n/ batch_size
validation_steps = generator_tuning.n / batch_size

## Class balance

In [ ]:
cls_train = generator_train.classes
cls_tuning = generator_tuning.classes

class_names = list(generator_train.class_indices.keys())
num_classes = generator_train.num_classes
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)
print(num_classes)
print(class_weight)
print(class_names)

# Build classification model

In [ ]:
def print_layer_trainable(model):
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
print_layer_trainable(model)
model = multi_gpu_model(model, gpus=2)

In [ ]:
optimizer = SGD(lr=0.05, momentum=0.9, decay=0.9)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train model

In [ ]:
for i in range(50):
    history = model.fit_generator(generator=generator_train,
                        epochs=1,          
                        steps_per_epoch=steps_per_epoch,
                        class_weight=class_weight,
                        validation_data=generator_tuning,
                        validation_steps=validation_steps)

    # save model every n step 
    if i % 5 == 0:
        model.save_weights(
            '/home/wf/model/tumor_normal_step%d_%.4f_%.4f_%.4f_%.4f.h5' 
            % (i, history.history['loss'][0], history.history['categorical_accuracy'][0], 
               history.history['val_loss'][0], history.history['val_categorical_accuracy'][0]))